Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

In [1]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir
# For download the models
!pip install huggingface_hub
!pip install datasets

'CMAKE_ARGS' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


We start by downloading an instruction-finetuned Mistral model, which we will ask to classify model outputs for us.

In [1]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Solar-10.7B-SLERP-GGUF"
model_basename = "solar-10.7b-slerp.Q4_K_M.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# This config has been tested on an RTX 3080 (VRAM of 16GB).
# you might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=12, # CPU cores
    n_batch=8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=64, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True
)

run_on_test = False # whether this baseline system is ran on the test splits or the val splits

llama_model_loader: loaded meta data with 25 key-value pairs and 435 tensors from C:\Users\tangl\.cache\huggingface\hub\models--TheBloke--Solar-10.7B-SLERP-GGUF\snapshots\e0d472af9430370775d9baa724f0f5eb2c0022ae\solar-10.7b-slerp.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 48
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 l

Running on the model-aware track data

In [2]:
# change paths appropriately
# make sure the output filename is the same as the reference filename for the scoring program
path_val_model_aware = "../raw/SHROOM_dev-v2/val.model-aware.v2.json"
path_val_model_aware_output = "./val.model-aware.json" 

from datasets import load_dataset
import json
import random
import numpy as np
import tqdm.notebook as tqdm
seed_val = 442
random.seed(seed_val)
np.random.seed(seed_val)

# simple JSON loading
with open(path_val_model_aware, 'r') as istr:
    data_val_all = json.load(istr)
num_sample = len(data_val_all)
print(num_sample)

output_json = []
labels = ["Not Hallucination", "Hallucination"]
"""
SelfCheckGPT Usage: (LLM) Prompt
https://github.com/potsawee/selfcheckgpt
Context: {}
Sentence: {}
Is the sentence supported by the context above?
Answer Yes or No:
"""
for i in tqdm.trange(num_sample):
    task = str(data_val_all[i]['task'])
    if run_on_test:
        # test splits will contain ids to ensure correct alignment before scoring
        id = int(data_val_all[i]['id'])
    hyp = str(data_val_all[i]['hyp'])
    src = str(data_val_all[i]['src'])
    tgt = str(data_val_all[i]['tgt'])

    if task == "PG":
        context = f"Context: {src}"
    else: #i.e. task == "MT" or task == "DM":
        context = f"Context: {tgt}"

    sentence = f"Sentence: {hyp}"
    message = f"{context}\n{sentence}\nIs the Sentence supported by the Context above? Answer using ONLY yes or no:"
    prompt = f"<s>[INST] {message} [/INST]"

    response = lcpp_llm(
        prompt=prompt,
        temperature= 0.0,
        logprobs=1,
    )
    answer = str(response["choices"][0]["text"]).strip().lower()
    if answer.startswith("yes"):
        output_label = "Not Hallucination"
        prob = 1-float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if answer.startswith("no"):
        output_label = "Hallucination"
        prob = float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if not answer.startswith("no") and not answer.startswith("yes"):
        idx_random = random.randint(0,len(labels)-1)
        output_label = labels[idx_random]
        prob = float(0.5)

    item_to_json = {"label":output_label, "p(Hallucination)":prob}
    if run_on_test:
        item_to_json['id'] = id
    
    output_json.append(item_to_json)


f = open(path_val_model_aware_output, 'w', encoding='utf-8')
json.dump(output_json, f)
f.close()
print("done")

501


  0%|          | 0/501 [00:00<?, ?it/s]


llama_print_timings:        load time =    1529.03 ms
llama_print_timings:      sample time =       0.84 ms /     4 runs   (    0.21 ms per token,  4773.27 tokens per second)
llama_print_timings: prompt eval time =    1528.91 ms /    51 tokens (   29.98 ms per token,    33.36 tokens per second)
llama_print_timings:        eval time =     353.81 ms /     3 runs   (  117.94 ms per token,     8.48 tokens per second)
llama_print_timings:       total time =    2026.49 ms /    54 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1529.03 ms
llama_print_timings:      sample time =       0.73 ms /     3 runs   (    0.24 ms per token,  4126.55 tokens per second)
llama_print_timings: prompt eval time =    1090.13 ms /    42 tokens (   25.96 ms per token,    38.53 tokens per second)
llama_print_timings:        eval time =     272.57 ms /     2 runs   (  136.28 ms per token,     7.34 tokens per second)
llama_print_timings:       total time =    1475.80 ms /    44 

done


Running on the model-agnostic track data

In [3]:
# change paths appropriately
# make sure the output filename is the same as the reference filename for the scoring program
path_val_model_agnostic = "../raw/SHROOM_dev-v2/val.model-agnostic.json"
path_val_model_agnostic_output = "./val.model-agnostic.json"

from datasets import load_dataset
import json
import random
import numpy as np
import tqdm.notebook as tqdm
seed_val = 442
random.seed(seed_val)
np.random.seed(seed_val)

# alternatively, one can use HuggingFace's library to load the data
dataset = load_dataset('json', data_files={'val':path_val_model_agnostic})
data_val_all = dataset['val']
num_sample = len(data_val_all)
print(dataset)
print(num_sample)

output_json = []
labels = ["Not Hallucination", "Hallucination"]
"""
SelfCheckGPT Usage: (LLM) Prompt
https://github.com/potsawee/selfcheckgpt
Context: {}
Sentence: {}
Is the sentence supported by the context above?
Answer Yes or No:
"""
for i in tqdm.trange(num_sample):
    # label = str(data_val_all[i]['label'])
    task = str(data_val_all[i]['task'])
    if run_on_test:
        # test splits will contain ids to ensure correct alignment before scoring
        id = int(data_val_all[i]['id'])
    hyp = str(data_val_all[i]['hyp'])
    src = str(data_val_all[i]['src'])
    tgt = str(data_val_all[i]['tgt'])

    if task == "PG":
        context = f"Context: {src}"
    else: #i.e. task == "MT" or task == "DM":
        context = f"Context: {tgt}"

    sentence = f"Sentence: {hyp}"
    message = f"{context}\n{sentence}\nIs the Sentence supported by the Context above? Answer using ONLY yes or no:"
    prompt = f"<s>[INST] {message} [/INST]"

    response = lcpp_llm(
        prompt=prompt,
        temperature= 0.0,
        logprobs=1,
    )
    answer = str(response["choices"][0]["text"]).strip().lower()
    if answer.startswith("yes"):
        output_label = "Not Hallucination"
        prob = 1-float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if answer.startswith("no"):
        output_label = "Hallucination"
        prob = float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if not answer.startswith("no") and not answer.startswith("yes"):
        idx_random = random.randint(0,len(labels)-1)
        output_label = labels[idx_random]
        prob = float(0.5)

    item_to_json = {"label":output_label, "p(Hallucination)":prob}
    if run_on_test:
        item_to_json['id'] = id
    output_json.append(item_to_json)


f = open(path_val_model_agnostic_output, 'w', encoding='utf-8')
json.dump(output_json, f)
f.close()
print("done")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating val split: 0 examples [00:00, ? examples/s]

DatasetDict({
    val: Dataset({
        features: ['label', 'src', 'labels', 'model', 'ref', 'task', 'tgt', 'p(Hallucination)', 'hyp'],
        num_rows: 499
    })
})
499


  0%|          | 0/499 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1529.03 ms
llama_print_timings:      sample time =       0.67 ms /     3 runs   (    0.22 ms per token,  4464.29 tokens per second)
llama_print_timings: prompt eval time =    1268.48 ms /    50 tokens (   25.37 ms per token,    39.42 tokens per second)
llama_print_timings:        eval time =     203.24 ms /     2 runs   (  101.62 ms per token,     9.84 tokens per second)
llama_print_timings:       total time =    1584.00 ms /    52 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1529.03 ms
llama_print_timings:      sample time =       0.61 ms /     3 runs   (    0.20 ms per token,  4934.21 tokens per second)
llama_print_timings: prompt eval time =     936.86 ms /    36 tokens (   26.02 ms per token,    38.43 tokens per second)
llama_print_timings:        eval time =     216.15 ms /     2 runs   (  108.08 ms per token,     9.25 tokens per second)
llama_print_timings:       to

done
